In [ ]:
# The code was removed by Watson Studio for sharing.

# Coursera Capstone Project
## The Battle of Neighborhoods (Part 1)
#### Coursera Capstone - REPORT CONTENT
1. Introduction Section : Discussion of the business problem and the interested audience in this project.
2. Data Section: Description of the data that will be used to solve the problem and the sources.
3. Methodology section: Discussion and description of exploratory data analysis carried out, any inferential statistical testing performed, and if any machine learnings were used establishing the strategy and purposes.
4. Results section: Discussion of the results.
5. Discussion section: Elaboration and discussion on any observations noted and any recommendations suggested based on the results.
6. Conclusion section: Report Conclusion.


## 1.) Intro

In [1]:
import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import os
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.21.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

### Scenario:
I am a beginner data scientist. I currently live with my parents in the suburbs of Chicago. I have been offered an opportunity to interview and work for a company in Manhattan, New York. I am very excited and I want to use this opportunity to practice my learnings in Coursera in order to be prepared for the area and for possible interview questions. The key question is : How can I find a convenient and enjoyable place similar to mine now in the suburbs? Certainly, I can use available real estate apps and Google but the idea is to use and apply the tools from the program. In order to make a comparison and evaluation of the rental options in Manhattan, NY. The apartment in Manhattan must meet the following demands:

1. Apartment must be 2 or 3 bedrooms.
2. Desired location is near a train Station within 1.0 mile (1.6 km) radius.
3. Price of rent not exceed $7,000 per month.
4. Top ammenities in the selected neighborhood would be cafes, entertainment centers, international restaurants, and grocery stores. 
5. Desirable to have venues such as cafes, restaurants, liquor stores, gyms, and grocery stores.

### Problem:
The challenge is to find a suitable apartment for rent in Manhattan, NY that complies with the demands on location, price and venues. The data required to resolve this challenge is described in the following section 2, below.
### Audience:
This gives information to, and helps, anyone planning on moving to a big city like Manhattan from less populous places in the US or the rest of the world. This also serves as practice in the development of a solid Data Science tool box of skills. 

## 2. Data
**Description of the data and its sources that will be used to solve the problem**

### Description:
The following data is required to answer the issues of the problem:

1. List of neighborhoods in Manhattan with their geodata (latitude and longitude).
2. List of train stations in Manhattan with their address.
3. List of apartments for rent in Manhattan area with their address and price.
4. Preferably, a list of apartment for rent with additional information, such as price, address, area, # of beds, etc.
5. Venues for each Manhattan neighborhood ( than can be clustered).
6. Venues for train stations, as needed.

### How the data will be used to solve the problem
The data will be used as follows:

1. Use Foursquare and geopy data to map top 10 venues for all Manhattan neighborhoods and clustered in groups.
2. Use foursquare and geopy data to map the location of train stations , separately and on top of the above clustered map in order to be able to identify the venues and ammenities near each station, or explore each subway location separately.
3. Use Foursquare and geopy data to map the location of rental places, in some form, linked to the subway location.
4. Create a map that depicts, for instance, the average rental price per square ft, around a radious of 1.0 mile (1.6 km) around each subway station. I will be able to quickly point to the popups to know the relative price per subway area.
5. Addresses from rental locations will be converted to geodata (lat,long) using Geopy-distance and Nominatim.
6. Data will be searched in open data sources if available, from real estate sites if open to reading, libraries or other government agencies.

**The procesing of this DATA will allow to answer the key questions to make a decision:**

    1. What is the cost of rent (per square ft) around a mile radius from each train station?
    2. What is the area of Manhattan with best rental pricing that meets criteria established?
    3. What is the distance from work to the tentative future home?
    4. What are the venues of the two best places to live? How the prices compare?
    5. How venues distribute among Manhattan neighborhoods and around train stations?
    6. Are there tradeoffs between size and price and location?
    7. Any other interesting statistical data findings of the real estate and overall data.

#### Reference of venues around current residence in the suburbs for comparison to Manhattan

In [2]:
address = 'Old Schaumburg Rd, Schaumburg, IL'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Illinois home is {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Illinois home is 42.0250138, -88.0424755.


In [3]:
lat = 42.0250138
long = -88.0424755
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 2000 # define radius
CLIENT_ID = 'SDLBIES51DQ32MTL22XEOPAIOPUJOW1E1NGRQ3JEVOJHOD0I'
CLIENT_SECRET = 'T3STUAGHODJ2VFMA2ZX0QRONFM52DALPHQRWKXPGDZENRXZC'
VERSION = '20180605'

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    long, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=SDLBIES51DQ32MTL22XEOPAIOPUJOW1E1NGRQ3JEVOJHOD0I&client_secret=T3STUAGHODJ2VFMA2ZX0QRONFM52DALPHQRWKXPGDZENRXZC&v=20180605&ll=42.0250138,-88.0424755&radius=2000&limit=100'

In [4]:
results = requests.get(url).json()

In [5]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [6]:
venues = results['response']['groups'][0]['items']
    
SGnearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SGnearby_venues =SGnearby_venues.loc[:, filtered_columns]

# filter the category for each row
SGnearby_venues['venue.categories'] = SGnearby_venues.apply(get_category_type, axis=1)

# clean columns
SGnearby_venues.columns = [col.split(".")[-1] for col in SGnearby_venues.columns]

SGnearby_venues.head(15)

name               categories        lat  \
0       Olympic Park Soccer Fields             Soccer Field  42.024142   
1                    Spring Valley          Nature Preserve  42.027111   
2                Shaw's Crab House       Seafood Restaurant  42.038123   
3                        Starbucks              Coffee Shop  42.039272   
4                         Benihana      Japanese Restaurant  42.039795   
5                       Seasons 52  New American Restaurant  42.038636   
6                         Wildfire               Steakhouse  42.040173   
7                             IHOP           Breakfast Spot  42.038749   
8               Whole Foods Market            Grocery Store  42.042192   
9                         creambar           Ice Cream Shop  42.040603   
10                       Starbucks              Coffee Shop  42.040215   
11              Jersey Mike's Subs           Sandwich Place  42.038675   
12     AMC Streets of Woodfield 20                Multiplex  42.039276   
13  Raising Cane's Chicken Fingers     Fast Food Restaurant  42.039431   
14                        Big Bowl       Chinese Restaurant  42.038865   

          lng  
0  -88.035389  
1  -88.051860  
2  -88.033179  
3  -88.047910  
4  -88.049686  
5  -88.036202  
6  -88.048348  
7  -88.037754  
8  -88.036541  
9  -88.033728  
10 -88.033610  
11 -88.047760  
12 -88.034721  
13 -88.049874  
14 -88.032951

## Map of Schaumburg with venues near home - for reference

In [7]:
map_sg = folium.Map(location = [latitude, longitude], zoom_start = 15)

# add markers to map
for lat, lng, label in zip(SGnearby_venues['lat'], SGnearby_venues['lng'], SGnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=4,
        radius=10,
        popup=label,
        color='blue',
        fill_color='blue',
        fill_opacity=0.7,
    ).add_to(map_sg)  
    
map_sg

## 3. Methodology section:
This section represents the main component of the report where the data is gathered, prepared for analysis. The tools described are used here and the Notebook cells indicates the execution of steps.

**The analysis and the stragegy:**

The strategy is based on mapping the above described data in section 2.0, in order to facilitate the choice of at least two candidate places for rent. The choice is made based on the demands imposed : location near a subway, rental price and similar venues to Singapore. This visual approach and maps with popups labels allow quick identification of location, price and feature, thus making the selection very easy.

The procesing of these DATA and its mapping will allow to answer the key questions to make a decision:

* What is the cost of available rental places that meet the demands?
* What is the cost of rent around a mile radius from each subway metro station?
* What is the area of Manhattan with best rental pricing that meets criteria established?
* What is the distance from work place ( Park Ave and 53 rd St) and the tentative future rental home?
* What are the venues of the two best places to live? How the prices compare?
* How venues distribute among Manhattan neighborhoods and around metro stations?
* Are there tradeoffs between size and price and location?
* Any other interesting statistical data findings of the real estate and overall data.

## METHODOLOY EXECUTION - Mapping Data

In [8]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

newyork_data
neighborhoods_data = newyork_data['features']
neighborhoods_data[0]
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()
pd.DataFrame.to_csv(manhattan_data, "manhattan_data")

address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    

CLIENT_ID = 'SDLBIES51DQ32MTL22XEOPAIOPUJOW1E1NGRQ3JEVOJHOD0I' # your Foursquare ID
CLIENT_SECRET = 'T3STUAGHODJ2VFMA2ZX0QRONFM52DALPHQRWKXPGDZENRXZC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

neighborhood_latitude = 40.7896239
neighborhood_longitude = -73.9598939

LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

results = requests.get(url).json()

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )
manhattan_venues.groupby('Neighborhood').count()

print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()

num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)



The geograpical coordinate of New York City are 40.7127281, -74.0060152.
The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.
Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards
There are 334 uniques categories.
----Battery Park City----
           venue  freq
0           Park  0.07
1    Coffee Shop  0.07
2          Hotel  0.05
3            Gym  0.05
4  Memorial Site  0.03


----Carnegie Hill----
         venue  freq
0  Coffee Shop  0.07
1  Pizza Place  0.04
2         Café  0.04
3  Yoga Studio  0.03
4    Bookstore 

## MANHATTAN NEIGHBORHOODS - DATA AND MAPPING

Cluster neighborhood data was produced with Foursquare during course lab work. A csv file was produced containing the neighborhoods around the 40 Boroughs. Now, the csv file is just read for convenience and consolidation of report.

In [9]:
#manhattan_data  = pd.read_csv('mh_neigh_data.csv') 
manhattan_data.head()

Borough        Neighborhood   Latitude  Longitude
0  Manhattan         Marble Hill  40.876551 -73.910660
1  Manhattan           Chinatown  40.715618 -73.994279
2  Manhattan  Washington Heights  40.851903 -73.936900
3  Manhattan              Inwood  40.867684 -73.921210
4  Manhattan    Hamilton Heights  40.823604 -73.949688

In [10]:
manhattan_data.tail()

Borough     Neighborhood   Latitude  Longitude
35  Manhattan       Turtle Bay  40.752042 -73.967708
36  Manhattan       Tudor City  40.746917 -73.971219
37  Manhattan  Stuyvesant Town  40.731000 -73.974052
38  Manhattan         Flatiron  40.739673 -73.990947
39  Manhattan     Hudson Yards  40.756658 -74.000111

## Manhattan Borough neighborhoods - data with top 10 clustered venues

In [11]:
#manhattan_merged = pd.read_csv('manhattan_merged.csv')
manhattan_merged.head()

Borough        Neighborhood   Latitude  Longitude  Cluster Labels  \
0  Manhattan         Marble Hill  40.876551 -73.910660               3   
1  Manhattan           Chinatown  40.715618 -73.994279               1   
2  Manhattan  Washington Heights  40.851903 -73.936900               1   
3  Manhattan              Inwood  40.867684 -73.921210               0   
4  Manhattan    Hamilton Heights  40.823604 -73.949688               0   

  1st Most Common Venue 2nd Most Common Venue 3rd Most Common Venue  \
0        Sandwich Place                   Gym           Coffee Shop   
1    Chinese Restaurant          Cocktail Bar   American Restaurant   
2                  Café         Grocery Store                Bakery   
3    Mexican Restaurant                Lounge                  Café   
4           Pizza Place                  Café           Coffee Shop   

   4th Most Common Venue 5th Most Common Venue 6th Most Common Venue  \
0             Donut Shop        Tennis Stadium       Supplement Shop   
1  Vietnamese Restaurant    Salon / Barbershop                   Spa   
2      Mobile Phone Shop    Mexican Restaurant    Spanish Restaurant   
3            Pizza Place            Restaurant    Spanish Restaurant   
4          Deli / Bodega    Mexican Restaurant           Yoga Studio   

  7th Most Common Venue 8th Most Common Venue 9th Most Common Venue  \
0            Steakhouse         Shopping Mall    Seafood Restaurant   
1                Bakery          Optical Shop     Hotpot Restaurant   
2        Sandwich Place    Chinese Restaurant           Supermarket   
3                Bakery    Chinese Restaurant    Frozen Yogurt Shop   
4  Caribbean Restaurant      Sushi Restaurant     Indian Restaurant   

  10th Most Common Venue  
0            Yoga Studio  
1         Ice Cream Shop  
2        Supplement Shop  
3                   Park  
4                   Park

## Map of Manhattan neighborhoods with top 10 clustered venues

### Allow us to identify each neighborhood and the cluster of venues around it in order to proceed to examine in more detail in the next cell

In [12]:
# create map of Manhattan using latitude and longitude values from Nominatim
latitude= 40.7308619
longitude= -73.9871558 

kclusters=5
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=20,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
  # add markers for rental places to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_clusters)    
    
       
map_clusters

### Examine a paticular Cluster - print venues

#### After examining several cluster datas, I concluded that cluster #_ resembles closer to Schaumburg, therefore providing guidance as to where to look for the future apartment.

#### Assign a value to 'kk' to explore a given cluster.

In [13]:
## kk is the cluster number to explore
kk = 2
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == kk, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

Neighborhood 1st Most Common Venue    2nd Most Common Venue  \
13  Lincoln Square                 Plaza       Italian Restaurant   
14         Clinton               Theater     Gym / Fitness Center   
24    West Village    Italian Restaurant  New American Restaurant   
39    Hudson Yards   American Restaurant       Italian Restaurant   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
13               Theater                  Café          Concert Hall   
14    Italian Restaurant   American Restaurant                   Spa   
24        Cosmetics Shop              Wine Bar   American Restaurant   
39  Gym / Fitness Center                  Café                 Hotel   

    6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
13  Performing Arts Venue  Gym / Fitness Center   American Restaurant   
14            Coffee Shop             Wine Shop                   Gym   
24                   Park          Cocktail Bar        Ice Cream Shop   
39            Coffee Shop                   Bar          Burger Joint   

   9th Most Common Venue 10th Most Common Venue  
13   Indie Movie Theater                   Park  
14                 Hotel         Sandwich Place  
24           Coffee Shop                Theater  
39                   Gym     Spanish Restaurant

## Map of Manhattan places for rent

### Several Manhattan real estate webs were webscrapped to collect rental data, as mentioned in section 2.0. The result was summarized in a csv file for direct reading, in order to consolidate the proces.

### The initial data for 144 apartment did not have the latitude and longitude data (NaN) but the information was established in the following cell using an algorythm and Nominatim.

In [22]:
# The code was removed by Watson Studio for sharing.

Address             Area  Price_per_ft2  Rooms  Area-ft2  \
0  West 105th Street  Upper West Side           2.94      5      3400   
1   East 97th Street  Upper East Side           3.57      3      2100   
2  West 105th Street  Upper West Side           1.89      4      2800   
3        CARMINE ST.     West Village           3.03      2      1650   
4    171 W 23RD ST.           Chelsea           3.45      2      1450   

   Rent_Price  Lat  Long  
0       10000  NaN   NaN  
1        7500  NaN   NaN  
2        5300  NaN   NaN  
3        5000  NaN   NaN  
4        5000  NaN   NaN

In [23]:
mh_rent = df_data_1
mh_rent.head()

Address             Area  Price_per_ft2  Rooms  Area-ft2  \
0  West 105th Street  Upper West Side           2.94      5      3400   
1   East 97th Street  Upper East Side           3.57      3      2100   
2  West 105th Street  Upper West Side           1.89      4      2800   
3        CARMINE ST.     West Village           3.03      2      1650   
4    171 W 23RD ST.           Chelsea           3.45      2      1450   

   Rent_Price  Lat  Long  
0       10000  NaN   NaN  
1        7500  NaN   NaN  
2        5300  NaN   NaN  
3        5000  NaN   NaN  
4        5000  NaN   NaN

In [26]:
mh_rent.tail()

Address                                Area  Price_per_ft2  \
139  200 East 72nd Street                Rental in Lenox Hill           5.15   
140      50 Murray Street            No fee rental in Tribeca           7.11   
141  300 East 56th Street       No fee rental in Midtown East           3.87   
142         1930 Broadway  No fee rental in Central Park West           5.06   
143    33 West 9th Street         Rental in Greenwich Village           6.67   

     Rooms  Area-ft2  Rent_Price  Lat  Long  
139      3      1700        8750  NaN   NaN  
140      2      1223        8700  NaN   NaN  
141      3      2100        8118  NaN   NaN  
142      2      1600        8095  NaN   NaN  
143      2      1500       10000  NaN   NaN

## Obtain geodata (lat,long) for each rental place in Manhattan with Nominatim
### Data was stored in a csv file for simplifaction report purposes and saving code processing time in future.

for n in range(len(mh_rent)):
address = mh_rent['Address'][n], address=(mh_rent['Address'][n]+ ' , '+' Manhattan NY '), geolocator = Nominatim(), location = geolocator.geocode(address), latitude = location.latitude, longitude = location.longitude, mh_rent['Lat'][n]=latitude, mh_rent['Long'][n]=longitude

#print(n,latitude,longitude)
time.sleep(2)

print('Geodata completed')
##Save dataframe to csv file
mh_rent.to_csv('MH_rent_latlong.csv',index=False) mh_rent.shape

In [33]:

body = client_f5d02fab24a44641a5c1673ee69d6238.get_object(Bucket='class9project-donotdelete-pr-vdosdf1jazohwv',Key='MH_rent_latlong.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_2 = pd.read_csv(body)
df_data_2.head()


Address             Area  Price_per_ft2  Rooms  Area-ft2  \
0  West 105th Street  Upper West Side           2.94      5      3400   
1   East 97th Street  Upper East Side           3.57      3      2100   
2  West 105th Street  Upper West Side           1.89      4      2800   
3        CARMINE ST.     West Village           3.03      2      1650   
4    171 W 23RD ST.           Chelsea           3.45      2      1450   

   Rent_Price        Lat       Long  
0       10000  40.799771 -73.966213  
1        7500  40.788585 -73.955277  
2        5300  40.799771 -73.966213  
3        5000  40.730523 -74.001873  
4        5000  40.744118 -73.995299

In [34]:
mh_rent=df_data_2
mh_rent.head()

Address             Area  Price_per_ft2  Rooms  Area-ft2  \
0  West 105th Street  Upper West Side           2.94      5      3400   
1   East 97th Street  Upper East Side           3.57      3      2100   
2  West 105th Street  Upper West Side           1.89      4      2800   
3        CARMINE ST.     West Village           3.03      2      1650   
4    171 W 23RD ST.           Chelsea           3.45      2      1450   

   Rent_Price        Lat       Long  
0       10000  40.799771 -73.966213  
1        7500  40.788585 -73.955277  
2        5300  40.799771 -73.966213  
3        5000  40.730523 -74.001873  
4        5000  40.744118 -73.995299

In [37]:
mh_rent.tail()

Address                                Area  Price_per_ft2  \
139  200 East 72nd Street                Rental in Lenox Hill           5.15   
140      50 Murray Street            No fee rental in Tribeca           7.11   
141  300 East 56th Street       No fee rental in Midtown East           3.87   
142         1930 Broadway  No fee rental in Central Park West           5.06   
143    33 West 9th Street         Rental in Greenwich Village           6.67   

     Rooms  Area-ft2  Rent_Price        Lat       Long  
139      3      1700        8750  40.769465 -73.960339  
140      2      1223        8700  40.714051 -74.009608  
141      3      2100        8118  40.758216 -73.965190  
142      2      1600        8095  40.772474 -73.981901  
143      2      1500       10000  40.733691 -73.997323

## Manhattan apartment rent price statistics

In [38]:
import seaborn as sns
sns.distplot(mh_rent['Rent_Price'],bins=15)

In [39]:
import seaborn as sns
sns.distplot(mh_rent['Price_per_ft2'],bins=15)

In [40]:
sns.boxplot(x='Rooms', y= 'Rent_Price', data=mh_rent)

## Map of Manhattan apartments for rent

In [41]:
# create map of Manhattan using latitude and longitude values from Nominatim
latitude= 40.7308619
longitude= -73.9871558

map_manhattan_rent = folium.Map(location=[latitude, longitude], zoom_start=12.5)

# add markers to map
for lat, lng, label in zip(mh_rent['Lat'], mh_rent['Long'],'$ ' + mh_rent['Rent_Price'].astype(str)+ ',  '+ mh_rent['Address']):      
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=6,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan_rent) 
    

map_manhattan_rent

## Map of Manhattan showing the places for rent and the cluster of venues

### Now, one can point to a rental place for price and address location information while knowing the cluster venues around it.
### This is an insightful way to explore rental possibilites

In [42]:
# create map of Manhattan using latitude and longitude values from Nominatim
latitude= 40.7308619
longitude= -73.9871558

# create map with clusters
kclusters=5
map_clusters2 = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=20,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters2)

# add markers to map for rental places
for lat, lng, label in zip(mh_rent['Lat'], mh_rent['Long'],'$ ' + mh_rent['Rent_Price'].astype(str)+ mh_rent['Address']):      
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=6,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_clusters2)  
    
    # Adds tool to the top right
from folium.plugins import MeasureControl
map_manhattan_rent.add_child(MeasureControl())

# FMeasurement ruler icon to establish distnces on map
from folium.plugins import FloatImage
url = ('https://media.licdn.com/mpr/mpr/shrinknp_100_100/AAEAAQAAAAAAAAlgAAAAJGE3OTA4YTdlLTkzZjUtNDFjYy1iZThlLWQ5OTNkYzlhNzM4OQ.jpg')
FloatImage(url, bottom=5, left=85).add_to(map_manhattan_rent)

map_clusters2

### Now one can explore a particular rental place and its venues in detail


In [43]:
## kk is the cluster number to explore
kk = 3
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == kk, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
0  Marble Hill        Sandwich Place                   Gym   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0           Coffee Shop            Donut Shop        Tennis Stadium   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0       Supplement Shop            Steakhouse         Shopping Mall   

  9th Most Common Venue 10th Most Common Venue  
0    Seafood Restaurant            Yoga Studio

## Mapping Manhattan Subway locations
### Manhattan subway metro locations ( address) was obtained from webscrapping sites such as Wikipedia, Google and NY Metro Transit. For simplification, a csv file was produced from the 'numbers' (Apple excel ) so that the reading of this file is the starting point here.
### The geodata will be obtain via Nominatim using the algorythm below.

In [45]:

body = client_f5d02fab24a44641a5c1673ee69d6238.get_object(Bucket='class9project-donotdelete-pr-vdosdf1jazohwv',Key='MH_subway_list.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_4 = pd.read_csv(body)
df_data_4.head()
body = client_f5d02fab24a44641a5c1673ee69d6238.get_object(Bucket='class9project-donotdelete-pr-vdosdf1jazohwv',Key='MH_subway.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_3 = pd.read_csv(body)
df_data_3.head()

sub_station                              sub_address  \
0  Dyckman Street Subway Station   170 Nagle Ave, New York, NY 10034, USA   
1       57 Street Subway Station                  New York, NY 10106, USA   
2                       Broad St                  New York, NY 10005, USA   
3             175 Street Station  807 W 177th St, New York, NY 10033, USA   
4                 5 Av and 53 St                  New York, NY 10022, USA   

         lat       long  
0  40.861857 -73.924509  
1  40.764250 -73.954525  
2  40.730862 -73.987156  
3  40.847991 -73.939785  
4  40.764250 -73.954525

In [46]:
# A csv file summarized the subway station and the addresses for next step to determine geodata
mh=df_data_4
mh.head()

sub_station                              sub_address
0  Dyckman Street Subway Station   170 Nagle Ave, New York, NY 10034, USA
1       57 Street Subway Station                  New York, NY 10106, USA
2                       Broad St                  New York, NY 10005, USA
3             175 Street Station  807 W 177th St, New York, NY 10033, USA
4                 5 Av and 53 St                  New York, NY 10022, USA

### Add colums labeled 'lat' and 'long' to be filled with geodata

In [47]:
#  Add columns 'lat'  and 'long' to mh dataframe - with random temporary numbers to get started
sLength = len(mh['sub_station'])
lat = pd.Series(np.random.randn(sLength))
long=pd.Series(np.random.randn(sLength))
mh = mh.assign(lat=lat.values)
mh = mh.assign(long=long.values)

## Algorythm to find latitude and longitud for each subway metro station and add them to dataframe
### This coding has been 'Markdown' just to simplify the file report, and the csv file will be read in cell below.

for n in range(len(mh)): address= mh['sub_address'][n] geolocator = Nominatim() location = geolocator.geocode(address) latitude = location.latitude longitude = location.longitude mh['lat'][n]=latitude mh['long'][n]=longitude

#print(n,latitude,longitude)
time.sleep(2)

print('Geodata completed')
##save dataframe to csv file
mh.to_csv('MH_subway.csv',index=False) mh.shape

### Read csv file that produced the subway stations list with geodata

In [48]:
mh=df_data_3
print(mh.shape)
mh.head()

(76, 4)


sub_station                              sub_address  \
0  Dyckman Street Subway Station   170 Nagle Ave, New York, NY 10034, USA   
1       57 Street Subway Station                  New York, NY 10106, USA   
2                       Broad St                  New York, NY 10005, USA   
3             175 Street Station  807 W 177th St, New York, NY 10033, USA   
4                 5 Av and 53 St                  New York, NY 10022, USA   

         lat       long  
0  40.861857 -73.924509  
1  40.764250 -73.954525  
2  40.730862 -73.987156  
3  40.847991 -73.939785  
4  40.764250 -73.954525

In [49]:
# removing duplicate rows and creating new set mhsub1
mhsub1=mh.drop_duplicates(subset=['lat','long'], keep="last").reset_index(drop=True)
mhsub1.shape

(22, 4)

In [50]:
mhsub1.tail()

sub_station                            sub_address  \
17   190 Street Subway Station   Bennett Ave, New York, NY 10040, USA   
18  59 St-Lexington Av Station     E 60th St, New York, NY 10065, USA   
19           57 Street Station      New York, NY 10019, United States   
20            14 Street / 8 Av      New York, NY 10014, United States   
21          MTA New York City   525 11th Ave, New York, NY 10018, USA   

          lat       long  
17  40.858113 -73.932983  
18  40.762259 -73.966271  
19  40.764250 -73.954525  
20  40.730862 -73.987156  
21  40.759809 -73.999282

## MAP of Manhattan showing the location of subway stations

In [51]:
# map subway stations 
# create map of Manhattan using latitude and longitude values obtain previoulsy via Moninatim geolocator
latitude=40.7308619
longitude=-73.9871558

map_mhsub1 = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers of subway locations to map
for lat, lng, label in zip(mhsub1['lat'], mhsub1['long'],  mhsub1['sub_station'].astype(str) ):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=6,
        radius=6,
        popup=label,
        color='red',
        fill_color='red',
        fill_opacity=2.5,
    ).add_to(map_mhsub1) 
map_mhsub1

## Map of Manhattan showing places for rent and the subway locations nearby
### Now, we can visualize the desirable rental places and their nearest subway station. Popups display rental address and monthly rental price and the subway station name.
### Notice that the icon in the top-right corner is a "ruler" that allows to measure the distance from a rental place to an specific subway station

In [52]:
mh_rent.head()

Address             Area  Price_per_ft2  Rooms  Area-ft2  \
0  West 105th Street  Upper West Side           2.94      5      3400   
1   East 97th Street  Upper East Side           3.57      3      2100   
2  West 105th Street  Upper West Side           1.89      4      2800   
3        CARMINE ST.     West Village           3.03      2      1650   
4    171 W 23RD ST.           Chelsea           3.45      2      1450   

   Rent_Price        Lat       Long  
0       10000  40.799771 -73.966213  
1        7500  40.788585 -73.955277  
2        5300  40.799771 -73.966213  
3        5000  40.730523 -74.001873  
4        5000  40.744118 -73.995299

In [53]:
# create map of Manhattan using latitude and longitude values from Nominatim
latitude= 40.7308619
longitude= -73.9871558

map_manhattan_rent = folium.Map(location=[latitude, longitude], zoom_start=13.3)

# add markers to map
for lat, lng, label in zip(mh_rent['Lat'], mh_rent['Long'],'$ ' + mh_rent['Rent_Price'].astype(str)+ mh_rent['Address']):      
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=6,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan_rent) 
    
    # add markers of subway locations to map
for lat, lng, label in zip(mhsub1['lat'], mhsub1['long'],  mhsub1['sub_station'].astype(str) ):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=6,
        radius=6,
        popup=label,
        color='red',
        fill_color='red',
        fill_opacity=2.5,
    ).add_to(map_manhattan_rent) 
    
    # Adds tool to the top right
from folium.plugins import MeasureControl
map_manhattan_rent.add_child(MeasureControl())

# Measurement ruler icon tool to measure distances in map
from folium.plugins import FloatImage
url = ('https://media.licdn.com/mpr/mpr/shrinknp_100_100/AAEAAQAAAAAAAAlgAAAAJGE3OTA4YTdlLTkzZjUtNDFjYy1iZThlLWQ5OTNkYzlhNzM4OQ.jpg')
FloatImage(url, bottom=5, left=85).add_to(map_manhattan_rent)

map_manhattan_rent

## 4. Results
## Map of Manhattan with rental places, subway locations and cluster of venues
### Red dots are Subway stations, Blue dots are apartments available for rent, Bubbles are the clusters of venues

In [54]:
# create map of Manhattan using latitude and longitude values from Nominatim
latitude= 40.7308619
longitude= -73.9871558

map_mh_one = folium.Map(location=[latitude, longitude], zoom_start=13.3)

# add markers to map
for lat, lng, label in zip(mh_rent['Lat'], mh_rent['Long'],'$ ' + mh_rent['Rent_Price'].astype(str)+ ', '+mh_rent['Address']):      
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=6,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_mh_one) 
    
    # add markers of subway locations to map
for lat, lng, label in zip(mhsub1['lat'], mhsub1['long'],  mhsub1['sub_station'].astype(str) ):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=6,
        radius=6,
        popup=label,
        color='red',
        fill_color='red',
        fill_opacity=2.5,
    ).add_to(map_mh_one) 


# set color scheme for the clusters
kclusters=5
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=15,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_mh_one)

    # Adds tool to the top right
from folium.plugins import MeasureControl
map_mh_one.add_child(MeasureControl())

# Measurement ruler icon tool to measure distances in map
from folium.plugins import FloatImage
url = ('https://media.licdn.com/mpr/mpr/shrinknp_100_100/AAEAAQAAAAAAAAlgAAAAJGE3OTA4YTdlLTkzZjUtNDFjYy1iZThlLWQ5OTNkYzlhNzM4OQ.jpg')
FloatImage(url, bottom=5, left=85).add_to(map_mh_one)

map_mh_one

## Problem Resolution - Select the apartment for rent
### The above consolidate map was used to explore options.
### After examining, I have chosen two locations that meet the requirements which will assess to make a choice.
* Apartment 1: 305 East 63rd Street in the Sutton Place Neighborhood and near 'subway 59th Street' station, Cluster # 2 Monthly rent : 7500 Dollars

* Apartment 2: 19 Dutch Street in the Financial District Neighborhood and near 'Fulton Street Subway' station, Cluster # 3 Monthly rent : 6935 Dollars

### Venues for Apartment 1 - Cluster 2

In [55]:
## kk is the cluster number to explore
kk = 2
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == kk, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

Neighborhood 1st Most Common Venue    2nd Most Common Venue  \
13  Lincoln Square                 Plaza       Italian Restaurant   
14         Clinton               Theater     Gym / Fitness Center   
24    West Village    Italian Restaurant  New American Restaurant   
39    Hudson Yards   American Restaurant       Italian Restaurant   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
13               Theater                  Café          Concert Hall   
14    Italian Restaurant   American Restaurant                   Spa   
24        Cosmetics Shop              Wine Bar   American Restaurant   
39  Gym / Fitness Center                  Café                 Hotel   

    6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
13  Performing Arts Venue  Gym / Fitness Center   American Restaurant   
14            Coffee Shop             Wine Shop                   Gym   
24                   Park          Cocktail Bar        Ice Cream Shop   
39            Coffee Shop                   Bar          Burger Joint   

   9th Most Common Venue 10th Most Common Venue  
13   Indie Movie Theater                   Park  
14                 Hotel         Sandwich Place  
24           Coffee Shop                Theater  
39                   Gym     Spanish Restaurant

### Venues for Apartment 2 - Cluster 3

In [56]:
## kk is the cluster number to explore
kk = 3
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == kk, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
0  Marble Hill        Sandwich Place                   Gym   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0           Coffee Shop            Donut Shop        Tennis Stadium   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0       Supplement Shop            Steakhouse         Shopping Mall   

  9th Most Common Venue 10th Most Common Venue  
0    Seafood Restaurant            Yoga Studio

## Apartment Selection
### Using the "one map" above, I was able to explore all possibilities since the popups provide the information needed for a good decision.
### Apartment 1 rent cost is US7500 slightly above the US7000 budget. Apt 1 is located 400 meters from subway station at 59th Street and work place ( Park Ave and 53rd) is another 600 meters way. I can walk to work place and use subway for other places aroung. Venues for this apt are as of Cluster 2 and it is located in a fine district in the East side of Manhattan.
### Apartment 2 rent cost is US6935, just under the US7000 budget. Apt 2 is located 60 meters from subway station at Fulton Street, but I will have to ride the subway daily to work , possibly 40-60 min ride. Venues for this apt are as of Cluster 3.
### Based on current Schaumburg area, I feel that Cluster 2 type of venues is a closer resemblance to my current place. That means that APARTMENT 1 is a better choice since the extra monthly rent is worth the conveniences it provides.

## 5. DISCUSSION
### In general, I am impressed with the overall organization, content and lab works presented during the Coursera IBM Certification Course
### I feel this Capstone project presented me a great opportunity to practice and apply the Data Science tools and methodologies learned.
### I have created a good project that I can use in my portfolio.
### I feel I have acquired a good starting point to become a professional Data Scientist and I will continue exploring to creating examples of practical cases.

## 6. CONCLUSIONS
### I feel rewarded with the effort, time and money spent. I believe this courses and all topics are necessary for any data scientist.
### This project has shown me a practical application to resolve a real situation that has a personal and financial impact using Data Science tools.
### The mapping with Folium is a very powerful technique to consolidate information and make a decision thoroughly and with confidence. I would recommend it in similar situations.